In [1]:
import pdfplumber
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials


In [35]:
# === 1. Extração dos dados da nota ===
with pdfplumber.open(r"C:\Users\abreu\Downloads\Notas de Corretagem_20260209.pdf") as pdf:
    tables = []
    for page in pdf.pages:
        for table in page.extract_tables():
            tables.extend(table)

# Definir colunas corretas
colunas = ["C/V", "Mercadoria", "Vencimento", "Quantidade", 
           "Preço/Ajuste", "Tipo do Negócio", "Valor Operação/Ajuste", 
           "D/C", "Taxa Operacional"]

df = pd.DataFrame(tables[1:], columns=colunas)

In [36]:
df.head()

,C/V,Mercadoria,Vencimento,Quantidade,Preço/Ajuste,Tipo do Negócio,Valor Operação/Ajuste,D/C,Taxa Operacional
0,C,BIT G26,27/02/2026,1,"360.080,0000",DAYTRADE,"97,20",C,"0,00"
1,C,BIT G26,27/02/2026,1,"360.240,0000",DAYTRADE,"95,60",C,"0,00"
2,V,BIT G26,27/02/2026,1,"360.460,0000",DAYTRADE,"93,40",D,"0,00"
3,C,BIT G26,27/02/2026,5,"360.700,0000",DAYTRADE,"455,00",C,"0,00"
4,V,BIT G26,27/02/2026,1,"361.620,0000",DAYTRADE,"81,80",D,"0,00"


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   C/V                    251 non-null    object
 1   Mercadoria             251 non-null    object
 2   Vencimento             251 non-null    object
 3   Quantidade             251 non-null    object
 4   Preço/Ajuste           251 non-null    object
 5   Tipo do Negócio        251 non-null    object
 6   Valor Operação/Ajuste  251 non-null    object
 7   D/C                    251 non-null    object
 8   Taxa Operacional       251 non-null    object
dtypes: object(9)
memory usage: 17.8+ KB


In [38]:
# === 2. Limpeza e conversão ===
def limpa_inteiro(valor):
    if not valor or pd.isna(valor):
        return 0
    return int(valor.replace(".", "").replace(",", "").strip())

def limpa_valor(valor):
    if not valor or pd.isna(valor):
        return 0.0
    return float(valor.replace(".", "").replace(",", ".").strip())

df["Quantidade"] = df["Quantidade"].apply(limpa_inteiro)

for col in ["Preço/Ajuste", "Valor Operação/Ajuste", "Taxa Operacional"]:
    df[col] = df[col].apply(limpa_valor)

df["Observações"] = ""

# === 3. Custos por contrato ===
custos = {"WIN G26": 1.50, "WDO H26": 0.90, "BIT G26": 0.50}
df["Custo Emolumentos"] = df["Mercadoria"].map(custos).fillna(0) * df["Quantidade"]

# Criar coluna de valor ajustado considerando compra/venda
df["Valor Ajustado"] = df.apply(
    lambda row: row["Valor Operação/Ajuste"] * (1 if row["C/V"] == "V" else -1),
    axis=1
)

# Resultado líquido = valor ajustado - custo
df["Resultado Líquido"] = df["Valor Ajustado"] - df["Custo Emolumentos"]

# IR só incide sobre lucro positivo
df["IR"] = df["Resultado Líquido"].apply(lambda x: x*0.20 if x > 0 else 0)


# === 4. Resultado e IR ===
df["Resultado Líquido"] = df["Valor Operação/Ajuste"] - df["Custo Emolumentos"]
df["IR"] = df["Resultado Líquido"].apply(lambda x: x*0.20 if x > 0 else 0)

# === 5. Painel consolidado ===
painel = df.groupby("Mercadoria").agg({
    "Quantidade":"sum",
    "Custo Emolumentos":"sum",
    "Resultado Líquido":"sum",
    "IR":"sum"
}).reset_index()


In [39]:
painel

,Mercadoria,Quantidade,Custo Emolumentos,Resultado Líquido,IR
0,,0,0.0,0.00,0.000
1,BIT G26,58,29.0,1928.41,385.682
2,WDO H26,54,48.6,3276.40,655.280
3,WIN G26,302,453.0,122464.00,24492.800


In [ ]:
import pdfplumber
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# === 1. Extração dos dados da nota ===
with pdfplumber.open("Notas de Corretagem_20260209.pdf") as pdf:
    tables = []
    for page in pdf.pages:
        for table in page.extract_tables():
            tables.extend(table)

colunas = ["C/V", "Mercadoria", "Vencimento", "Quantidade", 
           "Preço/Ajuste", "Tipo do Negócio", "Valor Operação/Ajuste", 
           "D/C", "Taxa Operacional"]

df = pd.DataFrame(tables[1:], columns=colunas)

# === 2. Limpeza e conversão ===
def limpa_inteiro(valor):
    if not valor or pd.isna(valor):
        return 0
    return int(valor.replace(".", "").replace(",", "").strip())

def limpa_valor(valor):
    if not valor or pd.isna(valor):
        return 0.0
    return float(valor.replace(".", "").replace(",", ".").strip())

df["Quantidade"] = df["Quantidade"].apply(limpa_inteiro)

for col in ["Preço/Ajuste", "Valor Operação/Ajuste", "Taxa Operacional"]:
    df[col] = df[col].apply(limpa_valor)

df["Observações"] = ""

# === 3. Custos por contrato ===
custos = {"WIN": 1.50, "WDO": 0.90, "BIT": 0.50}
df["Custo Emolumentos"] = df["Mercadoria"].map(custos).fillna(0) * df["Quantidade"]

# === 4. Resultado e IR ===
df["Resultado Líquido"] = df["Valor Operação/Ajuste"] - df["Custo Emolumentos"]
df["IR"] = df["Resultado Líquido"].apply(lambda x: x*0.20 if x > 0 else 0)

# === 5. Painel consolidado ===
painel = df.groupby("Mercadoria").agg({
    "Quantidade":"sum",
    "Custo Emolumentos":"sum",
    "Resultado Líquido":"sum",
    "IR":"sum"
}).reset_index()

# === 6. Integração com Google Sheets ===
scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name("credenciais.json", scope)
client = gspread.authorize(creds)

sheet = client.open("Controle Financeiro Fpleme").sheet1
sheet.update([df.columns.values.tolist()] + df.values.tolist())

try:
    painel_sheet = client.open("Controle Financeiro Fpleme").worksheet("Painel")
except:
    painel_sheet = client.open("Controle Financeiro Fpleme").add_worksheet(title="Painel", rows="100", cols="20")

painel_sheet.update([painel.columns.values.tolist()] + painel.values.tolist())
